### Install and Import libraries for this project

In [4]:
!pip install -r requirements.txt

In [110]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib
import re
import nltk
import numpy
import pickle
import warnings
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import swadesh
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('swadesh')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/gmondaut/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gmondaut/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /Users/gmondaut/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package swadesh to
[nltk_data]     /Users/gmondaut/nltk_data...
[nltk_data]   Package swadesh is already up-to-date!


### Load the data

In [111]:
def load_data(vacancies_data, professioni_data):
    ''' load datasets'''
    vacancies = pd.read_csv(vacancies_data)
    vacancies['Role'] = vacancies['Role'].replace('Commessa','commesso di vendita')
    vacancies['Role'] = vacancies['Role'].replace('Tecnico elettromeccanico','Manutentore elettromeccanico')
    professioni = pd.read_csv(professioni_data)
    return vacancies, professioni

In [112]:
vacancies, professioni = load_data(vacancies_data = 'train.csv', professioni_data = 'Professioni.csv')

In [113]:
print('Dataset with Source, Job Description and Role')
print(vacancies.shape)
vacancies.head()

Dataset with Source, Job Description and Role
(14, 3)


,Fonte,Job_Description,Role
0,Randstad Technical,Randstad Technical di Vimercate seleziona un m...,Manutentore elettromeccanico
1,Monster,Ricerchiamo N. 5 ricerca ADDETTI/E VENDITA per...,commesso di vendita
2,Monster,"Quanta SpA, Filiale di Verona, seleziona per a...",Manutentore elettromeccanico
3,Monster,"Kelly Services, multinazionale americana leade...",Manutentore elettromeccanico
4,Monster,ADHR Filiale di Torino ricerca per azienda cli...,Data Scientist


In [114]:
print('Dataset providing more information about Roles and skills')
print(professioni.shape)
professioni.head()

Dataset providing more information about Roles and skills
(4, 10)


,Group,Subgroup,Descrizione_professione,Descrizione_professione2,subgroup1,subgroup2,Esco_Abilit_compet_essenz,Esco_conoscenze_essenz,Esco_Abilit_compet_opz,Conoscenze
0,Statistici,Attuario\nData scientist\nEsperto in ricerca o...,Accertamenti tecnici per la valutazione delle ...,Elaborare piani tecnici per la trasformazione ...,consulente attuariale\ndemografo demografa\nsp...,consulente attuariale\ndemografo demografa\nsp...,correggere i dati\ncostruire sistemi di raccom...,classificazione delle informazioni\nelaborazio...,creare modelli di dati\ndefinire criteri di qu...,LDAP\nLINQ\nMDX\nN1QL\nSPARQL\nXQuery\nbusines...
1,Cuochi in alberghi e ristoranti,Aiuto cuoco\nAiuto cuoco di ristorante\nAntipa...,Allestimento dei piatti\nApplicazione delle pr...,Effettuare la preparazione dell'impasto da piz...,capo chef pasticcere capo chef pasticcera\nche...,addetta alla preparazione degli alimenti\ncuoc...,consegnare la zona di preparazione di alimenti...,"legislazione in materia di salute, sicurezza e...",NaN,NaN
2,Commessi alle vendite al minuto,Addetto al banco vendita di macelleria\nAddett...,Allestimento degli scaffali e delle isole prom...,Allestire e rifornire gli scaffali e le isole ...,commesso di negozio di abbigliamento commessa ...,commessa di negozio di abbigliamento uomo donn...,applicare le competenze matematiche\nconsiglia...,argomentazione di vendita\ncaratteristiche dei...,eseguire piccoli interventi di sartoria,industria dell'abbigliamento
3,Elettromeccanici,Aggiustatore elettromeccanico\nAvvolgitore di ...,Cablatura dell'impianto elettrico\nCollaudo de...,Effettuare la diagnosi tecnica (strumentale e ...,manutentore elettromeccanico manutentrice elet...,elettricista a bordo macchina\nelettricista-me...,adattare le attrezzature meccanizzate\napplica...,elettricità\nelettromeccanica\nelettronica\nme...,applicare un rivestimento alle apparecchiature...,accessori per cavi elettrici\ncomponenti di ba...


### Text Preprocessing on Professioni

In [115]:
def readable_format(dataset):
    ''' convert data into a readable format '''
    dataset.Subgroup = dataset.Subgroup.apply(lambda x: x.lower().split('\n'))
    dataset.Subgroup = dataset.Subgroup.apply(lambda x: [i.split("'") for i in x])

    dataset.subgroup1 = dataset.subgroup1.apply(lambda x: x.lower().split('\n'))
    dataset.subgroup1 = dataset.subgroup1.apply(lambda x: [i.split("'") for i in x])

    dataset.subgroup2 = dataset.subgroup2.apply(lambda x: x.lower().split('\n'))
    dataset.subgroup2 = dataset.subgroup2.apply(lambda x: [i.split("'") for i in x])

    dataset.Subgroup = dataset.Subgroup + professioni.subgroup1 + professioni.subgroup2
    dataset.Subgroup  = dataset.Subgroup.apply(lambda x: [ i for t in x for i in t])
    dataset.Subgroup = dataset.Subgroup.apply(lambda x: list(set(x)))
    
    # the following removes empty space from the subgroup list of values
    for i in range(0,dataset.Subgroup.shape[0]):
        while("" in dataset.Subgroup.iloc[i]) : 
            dataset.Subgroup.iloc[i].remove("") 
    
    return dataset

def list_of_skills(dataset):
    ''' to get list of skills from professioni dataset'''
    dataset1 = dataset.copy()
    dataset1.Conoscenze = dataset1.Conoscenze.apply(lambda x: str(x).lower().split('\n'))
    dataset1.Conoscenze = dataset1.Conoscenze.apply(lambda x: [i.split("'") for i in x])
    dataset1.Conoscenze = dataset1.Conoscenze.apply(lambda x: [i for t in x for i in t])
    Conoscenze_lista = []
    for i in dataset1.Conoscenze:
        for item in i:
            Conoscenze_lista.append(item)
    return Conoscenze_lista


def get_role_info(dataset, Conoscenze_lista):
    ''' create a column which capture role description '''
    dataset.Descrizione_professione = dataset.Descrizione_professione.apply(lambda x: x.lower().split('\n'))
    dataset.Descrizione_professione = dataset.Descrizione_professione.apply(lambda x: [i.split("'") for i in x])

    dataset.Descrizione_professione2 = dataset.Descrizione_professione2.apply(lambda x: x.lower().split('\n'))
    dataset.Descrizione_professione2 = dataset.Descrizione_professione2.apply(lambda x: [i.split("'") for i in x])

    dataset.Conoscenze = dataset.Conoscenze.apply(lambda x: str(x).lower().split('\n'))
    dataset.Conoscenze = dataset.Conoscenze.apply(lambda x: [i.split("'") for i in x])
    dataset['Descrizione_all'] = dataset.Descrizione_professione + professioni.Descrizione_professione2 + professioni.Conoscenze
    dataset['Descrizione_all'] = dataset.Descrizione_all .apply(lambda x: [ i for t in x for i in t])
    dataset['Descrizione_all']= dataset.Descrizione_all.apply(lambda x: [i.split("/") for i in x])
    dataset['Descrizione_all'] = dataset.Descrizione_all .apply(lambda x: [ i for t in x for i in t])
    
    dataset['Descrizione_all'] = dataset.Descrizione_all .apply(lambda x: list(set(x)))
    
    # to remove empty space
    for i in range(0,dataset.Descrizione_all.shape[0]):
        while("" in dataset.Descrizione_all.iloc[i]) : 
            dataset.Descrizione_all.iloc[i].remove("") 
    # remove punctuation and tokeniz
    dataset.Descrizione_all = dataset.Descrizione_all.apply(lambda x: list(set([t for i in x for t in \
                                                                                word_tokenize(i)])))
    
    dataset.Descrizione_all = dataset.Descrizione_all.apply(lambda x: [i for i in x if (len(i) > 3 or \
                                                                                    i in Conoscenze_lista)])
    
    dataset.Descrizione_all = dataset.Descrizione_all.apply(lambda x: [i for i in x if not (i.startswith('nan') or\
                                                                    i.startswith('dell') \
                                                                    or i.startswith('sull')\
                                                                     or i.startswith('degl')
                                                                     or i.startswith('dall')
                                                                     or i.startswith('all'))])
    return dataset


def get_info_to_use(dataset, column):
    ''' to export info into a csv file'''
    info_data = dataset[column]
    info_data.Conoscenze = info_data.Conoscenze.apply(lambda x: [i for t in x for i in t])
    info_data.Conoscenze = info_data.Conoscenze.apply(lambda x: ' '.join(x))
    info_data.Descrizione_all = info_data.Descrizione_all.apply(lambda x: ' '.join(x))
    info_data['info'] = info_data['Descrizione_all'] + ' ' + info_data['Conoscenze']
    info_data['info'] = info_data['info'].apply(lambda x: [i for i in word_tokenize(x)])
    info_data.drop(['Conoscenze','Descrizione_all'], axis = 1, inplace = True)
    info_data['info'] = info_data['info'].apply(lambda x: [i for i in x if (len(i) > 3 or i in Conoscenze_lista)])

    info_data['info']= info_data['info'].apply(lambda x: [i for i in x if not (i.startswith('loro') or\
                                                                           i.startswith('nan') or\
                                                                    i.startswith('dell') \
                                                                    or i.startswith('sull')\
                                                                     or i.startswith('degl')
                                                                     or i.startswith('dall')
                                                                     or i.startswith('all')
                                                                          )])
    info_data.to_csv('info_data.csv')
    return info_data

def remove_uncommon_it_words(dataset):
    ''' to get rid of uncommon words '''
    uncommon_it_words = swadesh.words('it')
    dataset['Descrizione_all'] = dataset['Descrizione_all'].apply(lambda x: [i for i in x if i not in uncommon_it_words])
    return dataset

In [116]:
# # uncommon ialian words 
# print(len(swadesh.words('it')))
# print(swadesh.words('it')[:20])
# uncommon_it_words = swadesh.words('it')


In [117]:
Conoscenze_lista = list_of_skills(professioni)
print('Lista delle Skills \n', Conoscenze_lista)
print(20 * '%')
professioni = readable_format(professioni)
professioni = get_role_info(professioni, Conoscenze_lista)
# professioni['Descrizione_all'] = remove_uncommon_it_words(professioni)
column = ['Conoscenze','Descrizione_all','Group']
info_data = get_info_to_use(professioni, column)
professioni = remove_uncommon_it_words(professioni)
professioni[['Group','Subgroup','Descrizione_all']].head()

Lista delle Skills 
 ['ldap', 'linq', 'mdx', 'n1ql', 'sparql', 'xquery', 'business intelligence', 'dati non strutturati', 'valutazione della qualità dei dati', 'nan', 'industria dell', 'abbigliamento', 'accessori per cavi elettrici', 'componenti di batterie', 'generatori elettrici', 'metodi di collaudo elettrico', 'piani di cablaggio elettrico', 'tecniche di brasatura', 'tecnologia per l', 'automazione']
%%%%%%%%%%%%%%%%%%%%


,Group,Subgroup,Descrizione_all
0,Statistici,"[statistico esperto in controlli di qualità, c...","[costruendo, domanda, metodologico, intelligen..."
1,Cuochi in alberghi e ristoranti,"[addetto alla preparazione della pizza, pastic...","[impiego, assegnati, innovando, ristorazione, ..."
2,Commessi alle vendite al minuto,"[addetto vendita di prodotti gastronomici, ven...","[impiego, picking, caratteristiche, antitacche..."
3,Elettromeccanici,"[elettromeccanici, riavvolgitore di motori ele...","[validazione, impiego, sostituiti, caratterist..."


#### Create a dictionary for professioni. e.g. 
{Statistici : ['attuario', 'data scientist', 'esperto in ricerca operativa',..... ]}

In [118]:
def dict_professioni(dataset):
    ''' to create a dictionary for professioni '''
    professioni_dictionary = pd.Series(dataset.Subgroup.values,index=dataset.Group).to_dict()
    # I need this step to convert back
    subgroup_dict = {}
    for group, subgroups in professioni_dictionary.items():
        for subgroup in subgroups:
            subgroup_dict[subgroup] = group
    
    return professioni_dictionary, subgroup_dict

In [119]:
professioni_dictionary, subgroup_dict = dict_professioni(professioni)
print('Professioni dictionary \n',list(professioni_dictionary.items())[0])
print(20 * '%')
print('Subgroup_dict \n',list(subgroup_dict.items())[:10])

Professioni dictionary 
 ('Statistici', ['statistico esperto in controlli di qualità', 'consulente attuariale', 'demografo demografa', 'esperto ricerca operativa', 'esperto in ricerca operativa', 'data scientist', 'statistici', 'statistico statistica', 'attuario', 'specialista in biometria', 'statistico applicato', 'statistico economico', 'statistico metodologico', 'statistico'])
%%%%%%%%%%%%%%%%%%%%
Subgroup_dict 
 [('statistico esperto in controlli di qualità', 'Statistici'), ('consulente attuariale', 'Statistici'), ('demografo demografa', 'Statistici'), ('esperto ricerca operativa', 'Statistici'), ('esperto in ricerca operativa', 'Statistici'), ('data scientist', 'Statistici'), ('statistici', 'Statistici'), ('statistico statistica', 'Statistici'), ('attuario', 'Statistici'), ('specialista in biometria', 'Statistici')]


### Text Preprocessing on vacancies

In [120]:
def text_preprocessing(dataset, Conoscenze_lista):
    ''' remove punctuation,noise and tokenize '''
    dataset.Job_Description = dataset.Job_Description.apply(lambda x: re.sub('[^a-zA-Z]',' ',x))
    dataset['Target'] = dataset['Role'].apply(lambda x: subgroup_dict.get(x.lower().strip(),'Unknown'))
    dataset.Target = dataset.Target.apply(lambda x: x.lower())
    dataset.Job_Description = dataset['Job_Description'].apply(lambda x: word_tokenize(x.lower())) # lowercase each word
    
    # to get rid of short words (len < 3) which are not skills
    dataset.Job_Description= dataset.Job_Description.apply(lambda x: [i for i in x if (len(i) > 3 \
                                                                                       or i in Conoscenze_lista)])
    
    # remove in 'job_description' words from 'fonte'
    dataset.Fonte = dataset.Fonte.apply(lambda x: word_tokenize(x))
    source = set([i.lower() for x in dataset.Fonte for i in x])
    dataset = dataset.drop('Fonte', axis = 1)
    dataset.Job_Description = dataset.Job_Description.apply(lambda x: [i for i in x if i not in source])
    
    # to remove italian stop words
    list_stopwords = list(set(stopwords.words('italian')))
    dataset.Job_Description = dataset.Job_Description.apply(lambda x: [i for i in x if i not in list_stopwords])    

    ## remove words like dell, della, dello, sull, degl. etc.
    dataset.Job_Description = dataset.Job_Description.apply(lambda x: [i for i in x if not (i.startswith('dell') \
                                                                        or i.startswith('sull')\
                                                                         or i.startswith('degl')
                                                                         or i.startswith('dall')
                                                                         or i.startswith('all')
                                                                       or i.strip().startswith('kell'))])
    return dataset

def remove_comuni(dataset, column, geo_dataset):
    ''' function to remove geographic noise '''
    
    geo_data = pd.read_fwf(geo_dataset)
    geo_data.to_csv('listacomuni.csv')
    geo_data = pd.read_csv('listacomuni.csv', sep=';')
    comune = [i.lower() for i in geo_data.Comune]
    geo_data.Provincia = geo_data.Provincia.apply(lambda x: str(x))
    provincia = [i.lower() for i in geo_data.Provincia]
    regione = [i.lower() for i in geo_data.Regione]
    
    dataset[column] = dataset[column].apply(lambda x: [i for i in x if i not in comune])
    dataset[column] = dataset[column].apply(lambda x: [i for i in x if i not in provincia])
    dataset[column] = dataset[column].apply(lambda x: [i for i in x if i not in regione])
    
    return dataset

In [121]:
vacancies = text_preprocessing(vacancies, Conoscenze_lista)
vacancies.head()

,Job_Description,Role,Target
0,"[vimercate, seleziona, manutentore, inseriment...",Manutentore elettromeccanico,elettromeccanici
1,"[ricerchiamo, ricerca, addetti, vendita, punto...",commesso di vendita,commessi alle vendite al minuto
2,"[filiale, verona, seleziona, azienda, cliente,...",Manutentore elettromeccanico,elettromeccanici
3,"[services, multinazionale, americana, leader, ...",Manutentore elettromeccanico,elettromeccanici
4,"[adhr, filiale, torino, ricerca, azienda, clie...",Data Scientist,statistici


In [122]:
vacancies = remove_comuni(vacancies, 'Job_Description', geo_dataset = 'listacomuni.txt')

In [123]:
def role_dictionary(dataset):
    ''' encode the role column '''
    Role_dictionary = pd.Series(dataset['Target'].unique()).to_dict()
    Role_dictionary = dict([(value, key) for key, value in Role_dictionary.items()])

    # save role dictionary as picke file
    with open('Role_dictionary.pickle', 'wb') as handle:
        pickle.dump(Role_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('Role_dictionary.pickle', 'rb') as handle:
        b = pickle.load(handle)

    role_encoded = []
    for i in dataset.Target:
        role_encoded.append(Role_dictionary.get(i, None))
    dataset.Role = role_encoded
    
    return dataset, Role_dictionary

In [124]:
vacancies,Role_dictionary = role_dictionary(vacancies)
vacancies.head()

,Job_Description,Role,Target
0,"[seleziona, manutentore, inserimento, somminis...",0,elettromeccanici
1,"[ricerchiamo, ricerca, addetti, vendita, punto...",1,commessi alle vendite al minuto
2,"[filiale, seleziona, azienda, cliente, settore...",0,elettromeccanici
3,"[services, multinazionale, americana, leader, ...",0,elettromeccanici
4,"[adhr, filiale, ricerca, azienda, cliente, set...",2,statistici


In [125]:
Role_dictionary

{'elettromeccanici': 0,
 'commessi alle vendite al minuto': 1,
 'statistici': 2,
 'cuochi in alberghi e ristoranti': 3,
 'unknown': 4}

###### Stemming / Lemmatization
Stemming:
Stemming algorithms work by cutting off the end of the word, and in some cases also the beginning while looking for the root. This indiscriminate cutting can be successful in some occasions, but not always, that is why we affirm that this an approach that offers some limitations. ex) studying -> study, studied -> studi

Lemmatization:
Lemmatization is the process of converting the words of a sentence to its dictionary form. For example, given the words amusement, amusing, and amused, the lemma for each and all would be amuse. ex) studying -> study, studied -> study. Lemmatization also discerns the meaning of the word by understanding the context of a passage. For example, if a "meet" is used as a noun then it will print out a "meeting"; however, if it is used as a verb then it will print out "meet".

We go for stemming

In [126]:
def stemming(vacancies, info_data):
    ''' stemming a text '''
    stemmer = nltk.stem.snowball.ItalianStemmer(ignore_stopwords=False)
    vacancies['Job_Description'] = vacancies['Job_Description'].apply(lambda x: [stemmer.stem(i) \
                                                                        for i in x])
    info_data['info'] = info_data['info'].apply(lambda x: [stemmer.stem(i) for i in x])
    return vacancies, info_data
    

In [127]:
vacancies, info_data = stemming(vacancies,info_data)
vacancies.head()

,Job_Description,Role,Target
0,"[selezion, manutentor, inser, somministr, pres...",0,elettromeccanici
1,"[ricerc, ricerc, addett, vend, punt, vend, abb...",1,commessi alle vendite al minuto
2,"[filial, selezion, azi, client, settor, metalm...",0,elettromeccanici
3,"[services, multinazional, american, leader, ri...",0,elettromeccanici
4,"[adhr, filial, ricerc, azi, client, settor, in...",2,statistici


In [128]:
info_data.head()

,Group,info
0,Statistici,"[costru, domand, metodolog, intelligenc, valut..."
1,Cuochi in alberghi e ristoranti,"[impieg, assegn, innov, ristor, pan, comp, foc..."
2,Commessi alle vendite al minuto,"[impieg, picking, caratterist, antitacchegg, f..."
3,Elettromeccanici,"[valid, impieg, sostitu, caratterist, monitor,..."


### Prepare data for training

In [129]:
#### Create one hot encoded columns for each word in info_data.Info
all_words = []
for i in info_data['info']:
    all_words.append(i)
lista_info_words = list(set([i for x in all_words for i in x]))

for i in lista_info_words:
    vacancies[i] = vacancies['Job_Description'].apply(lambda x:  1 if i in x else 0)

vacancies['Job_Description'] = vacancies.Job_Description.apply(lambda x: ' '.join(x))
print(vacancies.shape)
vacancies.head()

(14, 434)


,Job_Description,Role,Target,termic,riscontr,domand,ristor,buffet,organizz,solvibil,...,metod,pul,rilev,funzion,propriet,metodolog,acquist,fornitur,pesatur,merc
0,selezion manutentor inser somministr press azi...,0,elettromeccanici,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,ricerc ricerc addett vend punt vend abbigl fas...,1,commessi alle vendite al minuto,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,filial selezion azi client settor metalmeccan ...,0,elettromeccanici,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,services multinazional american leader risors ...,0,elettromeccanici,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,adhr filial ricerc azi client settor informat ...,2,statistici,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
X = vacancies.copy()
X.drop(['Role','Target'], axis = 1, inplace = True)
y = vacancies['Role']
X.head()
X_train, X_test, y_train, y_test = train_test_split(X, y)
print('Number of rows in the total set: {}'.format(vacancies.shape))
print('Number of rows in the training set: {}'.format(X_train.shape))
print('Number of rows in the test set: {}'.format(X_test.shape))

Number of rows in the total set: (14, 434)
Number of rows in the training set: (10, 432)
Number of rows in the test set: (4, 432)


#### Bag of Words processing to our dataset
cv = CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))

##### max_df:
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). This is to ensure that we only have words relevant to the context and not commonly used words.

##### max_features:
determines the number of columns in the matrix.

##### n-gram
range: we would want to look at a list of single words, two words (bi-grams) and three words (tri-gram) combinations.

count_vector = CountVectorizer(max_df = 0.8, ngram_range=(1,3))

In [131]:
count_vector = CountVectorizer(max_df = 0.8, ngram_range=(1,3))

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train['Job_Description']).toarray()

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test['Job_Description']).toarray()

training_data.shape

(10, 1169)

In [132]:
training_data = numpy.concatenate((training_data, X_train.iloc[:,1:].to_numpy()), axis = 1)
testing_data = numpy.concatenate((testing_data, X_test.iloc[:,1:].to_numpy()), axis = 1)

In [133]:
print('training data',training_data.shape)
print('testing data',testing_data.shape)

training data (10, 1600)
testing data (4, 1600)


### Training and Prediction

In [134]:
def training(model, training_data, y_train):
    ''' fit the model '''
    naive_bayes = model
    naive_bayes.fit(training_data, y_train)
    return naive_bayes

def get_key(value, dictionary):
    for k,v in dictionary.items():
        if value == v:
            return k

def prediction(testing_data):
    model = training(MultinomialNB(), training_data, y_train)
    predictions = model.predict(testing_data)
    predictions_keys = []
    for i in predictions:
        predictions_keys.append(get_key(i,Role_dictionary))
    print('Prediction for rows {} are \n {}'.format(X_test.index.tolist(),predictions_keys))
    print(100*'%')
    actual_predictions = []
    for i in y_test.tolist():
        actual_predictions.append(get_key(i,Role_dictionary))
    print('actual predictions are: \n {}'.format(actual_predictions))
    print(10*'%')
    print('Accuracy score: ', format(accuracy_score(y_test,predictions)))
    return predictions

In [135]:
predictions = prediction(testing_data)

Prediction for rows [5, 3, 11, 9] are 
 ['statistici', 'elettromeccanici', 'statistici', 'commessi alle vendite al minuto']
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
actual predictions are: 
 ['statistici', 'elettromeccanici', 'statistici', 'commessi alle vendite al minuto']
%%%%%%%%%%
Accuracy score:  1.0


#### Support Vector Machine

In [136]:
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo', probability=True)
clf.fit(training_data, y_train)
dec = clf.predict(testing_data)
svm_predictions_labels = []
for i in dec:
    svm_predictions_labels.append(get_key(i,Role_dictionary))
svm_predictions_labels

['cuochi in alberghi e ristoranti',
 'elettromeccanici',
 'cuochi in alberghi e ristoranti',
 'elettromeccanici']